In [1]:
import pandas as pd
from scipy.stats import linregress
import numpy as np
import geopandas as gpd
from pyproj import Transformer
from shapely.geometry import Polygon
import branca.colormap
import folium
import contextily as ctx
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from datetime import datetime
import matplotlib.dates as mdates
from matplotlib.ticker import FormatStrFormatter
import streamlit as st
import plotly.express as px
import pandas as pd


# Specify the path to your CSV file

file_path = '/mnt/d/Optimaize/Cantal/EGMS_L3_E37N24_100km_U/EGMS_L3_E37N24_100km_U.csv'

# Use pandas to read the CSV file into a DataFrame
df = pd.read_csv(file_path)

transformer = Transformer.from_crs('epsg:3035', 'epsg:4326', always_xy=True)

# Perform the transformation
coords = df.apply(
    lambda row: transformer.transform(row['easting'], row['northing']), axis=1
)

# Assign the transformed coordinates back to the original DataFrame using loc
for idx, (lon, lat) in coords.iteritems():
    df.loc[idx, 'longitude'] = lon
    df.loc[idx, 'latitude'] = lat


# Assuming high_r_squared_positive_df is your DataFrame with longitude and latitude
# Convert DataFrame to GeoDataFrame
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df.longitude, df.latitude),
    crs="EPSG:4326"  # Set the coordinate reference system to WGS84 (lat/lon)
)


# Assuming gdf is your GeoDataFrame loaded with your data

# Function to create a scatter mapbox with Plotly
def create_interactive_map(gdf):
    fig = px.scatter_mapbox(gdf,
                            lat="latitude",
                            lon="longitude",
                            # Assuming you have a 'mean_velocity' column to color the points
                            color="mean_velocity",
                            size="mean_velocity_std",
                            hover_name=gdf.index, # show index in hover data
                            zoom=3,
                            height=300)
    fig.update_layout(mapbox_style="open-street-map")
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    return fig

# Streamlit layout
st.title('Ground Displacement Interactive Map')

# Display interactive map in Streamlit
fig = create_interactive_map(gdf)
st.plotly_chart(fig, use_container_width=True)

# Instructions for user interaction
st.write("Click on a point in the map to view displacement time series.")

# Mock function to get time series data for a selected point
def get_time_series_data(selected_point):
    # You would fetch the real time series data based on the selected point
    # Here, we just return a sample Pandas series
    dates = pd.date_range(start="2020-01-01", periods=100, freq='W')
    data = pd.Series(data=range(100), index=dates)
    return data

# User selects a point ID (you could create a more dynamic selection method)
selected_point = st.selectbox('Select a Point ID', gdf.index)

# Show time series for selected point
if st.button('Show Time Series'):
    ts_data = get_time_series_data(selected_point)
    ts_fig = px.line(ts_data, title='Displacement over Time')
    st.plotly_chart(ts_fig, use_container_width=True)

KeyboardInterrupt: 